In [1]:
import nbimporter
import tensorflow as tf
from tensorflow import keras
import pickle
import statistics
from sklearn import preprocessing
from Top_Model_Generator_custom_loss_V5 import mae, custom_loss, mass_lambda, deltaeta_lambda
from GraphFuncs import*
import os
import math

Importing Jupyter notebook from Top_Model_Generator_custom_loss_V5.ipynb
Welcome to JupyROOT 6.22/02


In [2]:
modelname='gen_nunubar_BDLSTM_1.0e-03_1.0e-04_256_40_1.0e-04_1.0e-04_1.0e-04_1'
datatype = 'k'

In [3]:
#Create folders for plots
try:
    os.mkdir("./Plots/%s" % modelname)
    os.mkdir("./Plots/%s/Model" % modelname)
    os.mkdir("./Plots/%s/Model/K" % modelname)
    os.mkdir("./Plots/%s/Model/EtaPhi" % modelname)
    os.mkdir("./Plots/%s/Comparison" % modelname)
    os.mkdir("./Plots/%s/Comparison/K" % modelname)
    os.mkdir("./Plots/%s/Comparison/EtaPhi" % modelname)
    os.mkdir("./Plots/%s/Reco" % modelname)
    os.mkdir("./Plots/%s/Reco/K" % modelname)
    os.mkdir("./Plots/%s/Reco/EtaPhi" % modelname)
except OSError:
    print("Folder %s already exists" % modelname)
    
model = keras.models.load_model('./models/%s.h5' % modelname,
                                custom_objects = {'mae' : mae, 
                                                  'custom_loss': custom_loss, 
#                                                   'momentumeq_lambda': momentumeq_lambda, 
                                                  'mass_lambda': mass_lambda,
                                                  'deltaeta_lambda': deltaeta_lambda})

#Get MC and ML data in Eta-Phi space
if datatype == 'k':
    yscaler=  pickle.load(open("yscaler","rb"))
    testin = pickle.load(open("c_gen_nunubartestin","rb"))
    testin = tf.convert_to_tensor(testin)
    val_pred = model.predict(testin)
    val_pred = val_pred.reshape(val_pred.shape[0], -1)
    val_pred = yscaler.inverse_transform(val_pred)
    nu_lorentz = lorentz_to_ptetaphim(val_pred[:, 0], 
                                      val_pred[:, 2], 
                                      val_pred[:, 4], 
                                      val_pred[:, 6]) 
    nubar_lorentz = lorentz_to_ptetaphim(val_pred[:, 1], 
                                         val_pred[:, 3], 
                                         val_pred[:, 5], 
                                         val_pred[:, 7])
    val_pred = np.array([nu_lorentz[0], 
                         nubar_lorentz[0],
                         nu_lorentz[1], 
                         nubar_lorentz[1], 
                         nu_lorentz[2], 
                         nubar_lorentz[2]])
    val_pred = val_pred.T
else:
    yscaler = pickle.load(open("etaphi_yscaler","rb"))
    testin = pickle.load(open("c_etaphi_testin","rb"))
    testin = tf.convert_to_tensor(testin)
    val_pred = model.predict(testin)
    val_pred = val_pred.reshape(val_pred.shape[0], -1)
    val_pred = yscaler.inverse_transform(val_pred)
    
recoval_pred = pickle.load(open("c_etaphi_reco_testout", "rb"))
testout = pickle.load(open("c_etaphi_testout", "rb"))
yscaler = pickle.load(open("etaphi_yscaler","rb"))
val_true = yscaler.inverse_transform(testout)
titles = ["Nu pT", "Nubar pT", "Nu eta", "Nubar eta", "Nu phi", "Nubar phi"]

Folder gen_nunubar_BDLSTM_1.0e-03_1.0e-04_256_40_1.0e-04_1.0e-04_1.0e-04_1 already exists


2021-05-12 21:08:34.421237: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-05-12 21:08:34.448483: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 1799995000 Hz
2021-05-12 21:08:34.448599: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55aa35393cd0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-12 21:08:34.448612: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [4]:
#Run plotting code with specific bins depending on kinematic variable
space = 'EtaPhi'
bins = 30
for i in range(len(val_pred.T)):
    if i < 2:
        lower = 0
        upper = 200
    elif i < 4:
        lower = -5
        upper = 5
    else:
        lower = -math.pi
        upper = math.pi

    preds = np.array([val_pred[:,i], recoval_pred[:,i]])
    
    prednames = ['ML Model', 'MC Reco']
    
    histo(modelname, 
          'Comparison', 
          space, 
          val_true[:,i], 
          preds, 
          titles[i], 
          prednames, 
          lower, 
          upper, 
          bins)
    
    RMSEcompare(modelname, 
                space, 
                val_true[:,i], 
                preds, 
                titles[i], 
                prednames, 
                lower, 
                upper, 
                bins)
    
    heatmap(modelname, 
            'Model', 
            space, 
            val_true[:,i], 
            val_pred[:,i], 
            titles[i], 
            lower, 
            upper, 
            bins)
    
    heatmap(modelname, 
            'Reco', 
            space, 
            val_true[:,i],
            recoval_pred[:,i], 
            titles[i], 
            lower, 
            upper, 
            bins)
    
    resmeans, fwhms, bincenters, samples = \
    plotgaussian(modelname, 
                 'Model', 
                 space, 
                 val_true[:,i], 
                 val_pred[:,i], 
                 titles[i], 
                 lower, 
                 upper, 
                 bins)
    
    recoresmeans, recofwhms, recobincenters, recosamples = \
    plotgaussian(modelname, 
                 'Reco', 
                 space, 
                 val_true[:,i], 
                 recoval_pred[:,i], 
                 titles[i], 
                 lower, 
                 upper, 
                 bins)
    
    scatter(modelname,
            space,
            [resmeans, recoresmeans], 
            [bincenters, recobincenters], 
            [samples, recosamples], 
            titles[i], 
            prednames, 
            'Mean')

In [5]:
print([resmeans, recoresmeans][1])

[-2.949013, -2.164975, -1.680387, -1.372606, -1.1052984, -0.86659, -0.64973706, -0.56447375, -0.43716243, -0.36472696, -0.28725946, -0.24584483, -0.0762462, -0.101866476, -0.077518746, 0.11830904, 0.09746323, 0.18952596, 0.21180251, 0.3127143, 0.3389633, 0.47211, 0.63548034, 0.7470331, 0.87640005, 1.1573416, 1.2811424, 1.6494613, 2.120837, 2.772686]


In [6]:
titles = ["Nu px","Nubar px","Nu py","Nubar py","Nu pz","Nubar pz","Nu E","Nubar E"]

In [7]:
#Get MC and ML data in momentum space
if datatype == 'k':
    yscaler = pickle.load(open("yscaler","rb"))
    testin = pickle.load(open("c_gen_nunubartestin","rb"))
    testin = tf.convert_to_tensor(testin)
    val_pred = model.predict(testin)
    val_pred = val_pred.reshape(val_pred.shape[0], -1)
    val_pred = yscaler.inverse_transform(val_pred)
else:
    testin = pickle.load(open("c_etaphi_testin","rb"))
    yscaler = pickle.load(open("etaphi_yscaler","rb"))
    testin = tf.convert_to_tensor(testin)
    val_pred = model.predict(testin)
    val_pred = val_pred.reshape(val_pred.shape[0], -1)
    val_pred = yscaler.inverse_transform(val_pred)
    nu_lorentz = ptetaphim_to_lorentz(val_pred[:,0], 
                                      val_pred[:,2], 
                                      val_pred[:,4], 0) 
    nubar_lorentz = ptetaphim_to_lorentz(val_pred[:,1], 
                                         val_pred[:,3], 
                                         val_pred[:,5], 0)
    val_pred = np.array([nu_lorentz[0], 
                         nubar_lorentz[0], 
                         nu_lorentz[1], 
                         nubar_lorentz[1], 
                         nu_lorentz[2], 
                         nubar_lorentz[2], 
                         nu_lorentz[3], 
                         nubar_lorentz[3]])
    val_pred = val_pred.T
    
testout = pickle.load(open("c_gen_nunubartestout","rb"))
yscaler = pickle.load(open("yscaler", "rb"))
val_true = yscaler.inverse_transform(testout)
recoval_pred = pickle.load(open("c_reco_nunubartest","rb"))

In [8]:
#Run plotting code with specific bins depending on kinematic variable
bins = 30
space = 'K'
for i in range(len(val_pred.T)):
    if i < 4:
        lower = -150
        upper = 150
    elif i < 6:
        lower = -400
        upper = 400
    else:
        lower = 0
        upper = 300
        
    preds = np.array([val_pred[:,i], recoval_pred[:,i]])
    
    prednames = ['ML Model', 'MC Reco']
    
    histo(modelname, 
          'Comparison', 
          space, 
          val_true[:,i], 
          preds, 
          titles[i], 
          prednames, 
          lower, 
          upper, 
          bins)
    
    RMSEcompare(modelname, 
                space, 
                val_true[:,i], 
                preds, 
                titles[i], 
                prednames, 
                lower, 
                upper, 
                bins)
    
    heatmap(modelname, 
            'Model', 
            space, 
            val_true[:,i], 
            val_pred[:,i], 
            titles[i], 
            lower, 
            upper, 
            bins)
    
    heatmap(modelname, 
            'Reco', 
            space, 
            val_true[:,i],
            recoval_pred[:,i], 
            titles[i], 
            lower, 
            upper, 
            bins)
    
    resmeans, fwhms, bincenters, samples = \
    plotgaussian(modelname, 
                 'Model', 
                 space, 
                 val_true[:,i], 
                 val_pred[:,i], 
                 titles[i], 
                 lower, 
                 upper, 
                 bins)
    
    recoresmeans, recofwhms, recobincenters, recosamples = \
    plotgaussian(modelname, 
                 'Reco', 
                 space, 
                 val_true[:,i], 
                 recoval_pred[:,i], 
                 titles[i], 
                 lower, 
                 upper, 
                 bins)
    
    scatter(modelname, 
            space,
            [resmeans, recoresmeans], 
            [bincenters, recobincenters], 
            [samples, recosamples], 
            titles[i], 
            prednames, 
            'Mean')